In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm
from datetime import datetime
from unidecode import unidecode
import warnings
import os
import random

sys.path.append(os.getcwd().replace("\\","/")[:-4])
import models
import functions as func
import paths as ph

sys.path.append(os.getcwd().replace("\\","/") + "/src")
import homologations as ho
import spider

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.49"}


actual_date = func.GetActualDate()
actual_year = datetime.today().year

paths_temp = paths.temp

ReadTemp = spider.ReadTemp
Criterios = spider.Criterios
RefineAtributes = spider.RefineAtributes
TrainModel = spider.TrainModel

tipo_de_moto = ho.tipo_de_moto

In [ ]:
#def Main(counter_file = 200, delay = 1, reprocess_oferts_urls = True): # Funcional delay = 1.5
print("Bloque 4: concatenado final de archivos")
df = ReadTemp()
df = df[df["estado"] == 1]
print("="*120)


print("Bloque 5: refinando atributos")
df = RefineAtributes(df)
print("-"*120)

print("Correctos final:", df[df["estado"] == 1].shape[0], "|", round((df[df["estado"] == 1].shape[0]/df.shape[0])*100, 2), "%")
print("Dimensión df:", df.shape)
vars_duplicates = ["marca", "modelo", "precio", "año", "cilindrada", "km", "barrio", "ciudad", "departamento"]
print("Duplicados por atributos:", df[df.duplicated(vars_duplicates)].shape[0])
print("Duplicados por url:", df[df.duplicated(["url"])].shape[0])
print("Duplicados por id:", df[df.duplicated(["id_ofert"])].shape[0])


df = df.drop_duplicates("url").drop_duplicates("id_ofert").drop_duplicates(vars_duplicates)
print("Dimensión df sin duplicados:", df.shape)

print("Bloque 6: imputación de datos faltantes en categoricas"); print("-"*50)
#df = models.ImputCategorics(df, "marca_modelo", "transmision", 70)
brands_ejecution = list(set(df["marca"].tolist()))
models_ejecution = list(set(df["modelo"].tolist()))
print("="*120)


In [ ]:
print(df.shape)
df = df[(df["año"].notna()) & (df["marca"].notna()) & (df["modelo"] != "Pendiente")]
print(df.shape)

In [4]:
df2 = df.copy()

In [ ]:
print(df["cilindrada"].isnull().sum() / df.shape[0])

In [6]:
df2 = df2[df2["cilindrada"] >= 60]
df = df2.copy()

In [ ]:
print(df.shape)
df = df[df["marca"].isin(ho.brands_homologue)]
print(df.shape)

print("Bloque 9: agrupacion de variables"); print("-"*50)
df = models.ClassicationTreeGroup(df, y = "precio", x = "marca", max_leaf_nodes=[10])
df = models.ClassicationTreeGroup(df, y = "precio", x = "modelo", max_leaf_nodes=[50])
print("="*120)

In [8]:

# print("Bloque 7: imputando valores faltantes en cilindraje"); print("-"*50)
# df["cilindrada"] = np.log(df["cilindrada"])
# df = models.ImputRegression(df, y = 'cilindrada', x = ['año', 'marca', 'modelo'], method = "lm")
# df["cilindrada"] = round(np.exp(df["cilindrada"]), 2)
# print("="*120)


In [ ]:
df[["precio", "año", "km", "cilindrada", "tipo_de_moto", "4_tiempos", 
    "km_por_año","marca", "modelo_agrup"]].dropna()

In [ ]:


print("TENIENDO EN CUENTA DUMMYS DE NULOS")


print("Bloque 8: criterios de exclusión e inclusión"); print("-"*50)
df = Criterios(df)
print("="*120)



df = df[['estado', 'nombre', 'precio', 'año', 'km', 'km_por_año', "tipo_de_moto", "4_tiempos", 
         'barrio', 'ciudad','departamento','fecha', 'url', 'dtm_etl', 'marca_modelo', 'marca', 
         'modelo', 'modelo_agrup','cilindrada']]

import paths as ph
save_model = True
#if save_model == True:
#df.to_csv(f"data/df_refine_{actual_date}.csv", sep = "|", index = False)
df.to_parquet(f'{ph.data_bike}/df_refine_{actual_date}.parquet', compression = "gzip")

dim_models_group = df[["modelo", "modelo_agrup"]].drop_duplicates()
func.pkl.Save(dim_models_group, f'{ph.pkl_bike}/motos_dim_models_group.pkl')
print("="*120)

print("Bloque 10: entrenando modelo predictivo"); print("-"*50)
model = TrainModel(df, save_model)
print("="*120)

print("---------- Precio ----------")
print("Media:", round(df["precio"].mean(), 2))
print("Mediana:", round(df["precio"].median(), 2))
print("CV:", round(np.sqrt(df["precio"].var()) / df["precio"].mean() * 100, 2), "%")
#print("Umbral:", round(umbral_precio, 3))
print("="*120)

print("Dimensión final:", df.shape)
#print(f"El proceso tardó: {datetime.now() - time1}")
print("="*120)


print("Marca")
print("','".join(list(set(brands_ejecution) - set(ho.brands_homologue))))
print("-"*120)
print("Modelo")
print("','".join(pd.Series(list(set(models_ejecution) - set(ho.models_all))).fillna("NULO")))
print("-"*120)


In [ ]:
cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
for col in cols_numeric:
    df[col] = df[col].astype(float)
df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", 
                "tipo_de_carroceria", "puertas","marca", "modelo_agrup"]].dropna()
df_train["precio"] = np.log(df_train["precio"].astype(float))

X_train, X_test, y_train, y_test = models.TrainTestDummies(df = df_train, y='precio',
                                                                dummies = ["marca", "modelo_agrup", "transmision", "tipo_de_combustible", 
                                                                        "tipo_de_carroceria", "puertas"])
print("Dimensión train:", X_train.shape)
print("Dimensión test:", X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

scalerX = MinMaxScaler()
scalerY = MinMaxScaler()
X_new_scaled = scalerX.transform(X)
y_pred_scaled = model.predict(X_new_scaled)
y_pred = scalerY.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_pred

In [ ]:
models.NN().predict(X_test)

# Entrenando NN (FUNCIONAL)

In [3]:
df = pd.read_parquet("/workspaces/cars/data/df_refine_20240915.parquet")
df.shape

cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
for col in cols_numeric:
    df[col] = df[col].astype(float)
df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", 
                "tipo_de_carroceria", "puertas","marca", "modelo_agrup"]].dropna()
df_train["precio"] = np.log(df_train["precio"].astype(float))

X_train, X_test, y_train, y_test = models.TrainTestDummies(df = df_train, y='precio',
                                                                dummies = ["marca", "modelo_agrup", "transmision", "tipo_de_combustible", 
                                                                        "tipo_de_carroceria", "puertas"])

In [ ]:
param_grid = {
                'layers': [[64, 128]], 
                'activation': ['relu'],      
                'optimizer': ['adam'],         
                'epochs': [100],                 
                'batch_size': [10]
            }
model, metrics_test = models.NN(X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test, param_grid = param_grid, cv = 2, save = True).fit()

# Predict

In [ ]:
año = 2005
km = 150000
motor = 4
marca = "Ford"
modelo = "Explorer"
tipo_de_carroceria = "Camioneta"
puertas = "4_5"
transmision = "Automático"
tipo_de_combustible = "Gasolina"
modelo = df[df["modelo"] == modelo]["modelo_agrup"].unique()[0]

import joblib
files_models = sorted(os.listdir("models"))[-1]
model = joblib.load("models/" + files_models)  

cols_numeric = ["precio", "año", "km", "motor", "km_por_año"]
for col in cols_numeric:
    df[col] = df[col].astype(float)
df_train = df[["precio", "año", "km", "motor", "km_por_año", "transmision", "tipo_de_combustible", "tipo_de_carroceria", "puertas",
                "marca", "modelo_agrup"]].dropna()
df_train["precio"] = np.log(df_train["precio"].astype(float))

dim_models = func.pkl.Load("pkl/dim_models_group.pkl")[["modelo", "modelo_agrup"]].drop_duplicates()
dim_doors = func.pkl.Load("pkl/dim_models_group.pkl")[["modelo", "puertas"]].drop_duplicates()

modelo = dim_models[dim_models["modelo"] == modelo]["modelo_agrup"].unique()[0]
puertas = dim_doors[dim_doors["modelo"] == modelo]["puertas"].unique().tolist()

if puertas == []:
    puertas = "4_5"
else:
    puertas = puertas[0]
df_new = pd.DataFrame({"año":[año], "km":[km], "motor":[motor], "km_por_año":[km/(datetime.today().year-año)], 
                        "transmision":[transmision], "tipo_de_combustible":[tipo_de_combustible],"marca":[marca], 
                        "modelo_agrup":[modelo], "puertas":[puertas], "tipo_de_carroceria":[tipo_de_carroceria]})
df_new = pd.get_dummies(df_new, columns = ["transmision", "tipo_de_combustible", "marca", "modelo_agrup",
                                            "puertas", "tipo_de_carroceria"]).replace(True, 1)

df_predict = pd.DataFrame(columns = model.feature_names_in_)
df_predict.loc[0] = 0
features = model.feature_names_in_.tolist()
features_delete = list(set(df_new.columns) - set(features))
if features_delete != []:
    df_predict.drop(columns = features_delete, inplace = True)

predict = int(np.exp(model.predict(features_delete))*1000000)